In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Урок 8. Рекуррентные нейронные сети RNN LSTM GRU
#### Домашнее задание
На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи

1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)
#### Подключение библиотек


In [5]:
!pip install pad_sequences

  Preparing metadata (setup.py) ... done
  Created wheel for pad_sequences: filename=pad_sequences-0.6.1-py3-none-any.whl size=10216 sha256=28ea4e57fee414228be811cf21beb11a8bc810b423a4197f8c8d6af0ce93b85b
  Stored in directory: /root/.cache/pip/wheels/c6/eb/b8/ff037500633aff1921fca1d194ee389b7300305fc56ed43e2e
Successfully built pad_sequences


In [17]:
!pip install conda

In [23]:
!pip install anaconda xlrd

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 5.1 MB/s eta 0:00:00
  Created wheel for anaconda: filename=anaconda-0.0.1.1-py3-none-any.whl size=1124 sha256=eab0853dc983abfad065ee032006a5f871cc2e88d34ab7851392648e2257b377
  Stored in directory: /root/.cache/pip/wheels/be/45/c0/24b80d62dbc86fcdd469270eca576e265cc1b25581623539af
Successfully built anaconda


In [27]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, GlobalMaxPool2D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping  
from keras.utils import np_utils

#### Загрузка и подготовка данных

In [28]:
df = pd.read_excel('/kaggle/input/feedback-summer/summer_feedback.xls')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [29]:
df = df.dropna()

In [31]:
df.loc[:, 'Content'] = df['Content'].astype(str)

In [32]:
X = df['Content']
y = df['Rating']

In [33]:
y = np.where(y == 5, 1, 0)

In [34]:
num_classes = 1
epochs = 20
batch_size = 512

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
text_corpus_train = X_train
text_corpus_test = X_test

In [37]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

In [38]:
max_words = word_count
max_len = training_length

### Построение CNN сети

In [39]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [40]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [41]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
30/30 [==============================] - 14s 422ms/step - loss: 0.5683 - accuracy: 0.6996 - val_loss: 0.4335 - val_accuracy: 0.8306
Epoch 2/20
30/30 [==============================] - 12s 408ms/step - loss: 0.3692 - accuracy: 0.8634 - val_loss: 0.3183 - val_accuracy: 0.8820


In [42]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

130/130 [==============================] - 1s 8ms/step
              precision    recall  f1-score   support

           0       0.79      0.74      0.76      1204
           1       0.90      0.92      0.91      2928

    accuracy                           0.87      4132
   macro avg       0.84      0.83      0.84      4132
weighted avg       0.86      0.87      0.87      4132



### Построение RNN

In [43]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [44]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [45]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
30/30 [==============================] - 11s 291ms/step - loss: 0.5446 - accuracy: 0.7496 - val_loss: 0.4335 - val_accuracy: 0.8179
Epoch 2/20
30/30 [==============================] - 9s 297ms/step - loss: 0.4346 - accuracy: 0.8219 - val_loss: 0.3847 - val_accuracy: 0.8512


In [46]:
preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

130/130 [==============================] - 2s 16ms/step
              precision    recall  f1-score   support

           0       0.73      0.64      0.68      1204
           1       0.86      0.90      0.88      2928

    accuracy                           0.83      4132
   macro avg       0.80      0.77      0.78      4132
weighted avg       0.82      0.83      0.82      4132



### Построение CNN+RNN

In [47]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))

model.build()

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 176, 128)          3158784   
                                                                 
 conv1d_1 (Conv1D)           (None, 174, 128)          49280     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 10)                1290      
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 3,209,365
Trainable params: 3,209,365
Non-trainable params: 0
____________________________________________

In [49]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3, activation='relu'))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.build()

print(model.summary())  

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 176, 128)          3158784   
                                                                 
 masking_2 (Masking)         (None, 176, 128)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 174, 128)          49280     
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 64)                12352     
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                

In [50]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

Epoch 1/20
30/30 [==============================] - 22s 661ms/step - loss: 0.4992 - accuracy: 0.7813 - val_loss: 0.3917 - val_accuracy: 0.8482
Epoch 2/20
130/130 [==============================] - 3s 21ms/step
              precision    recall  f1-score   support

           0       0.83      0.68      0.74      1204
           1       0.88      0.94      0.91      2928

    accuracy                           0.86      4132
   macro avg       0.85      0.81      0.83      4132
weighted avg       0.86      0.86      0.86      4132



### Построение CNN+MaxPool+RNN

In [51]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(128, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPool1D(keepdims=True)) # keepdims Сохраняет размерность для SimpleRNN
model.add(SimpleRNN(64))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.build()

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 176, 128)          3158784   
                                                                 
 masking_3 (Masking)         (None, 176, 128)          0         
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 176, 128)          32896     
                                                                 
 conv1d_4 (Conv1D)           (None, 174, 128)          49280     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 1, 128)           0         
 balMaxPooling1D)                                                
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 64)                12352     
                                                      

In [52]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

Epoch 1/20
30/30 [==============================] - 35s 1s/step - loss: 0.5160 - accuracy: 0.7749 - val_loss: 0.4244 - val_accuracy: 0.8294
Epoch 2/20
130/130 [==============================] - 5s 33ms/step
              precision    recall  f1-score   support

           0       0.80      0.67      0.73      1204
           1       0.87      0.93      0.90      2928

    accuracy                           0.86      4132
   macro avg       0.84      0.80      0.82      4132
weighted avg       0.85      0.86      0.85      4132



### Построение RNN+CNN+RNN

In [53]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3, activation='relu'))
model.add(SimpleRNN(128, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(64, 3, activation='relu'))
model.add(SimpleRNN(64, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(Conv1D(32, 2, activation='relu'))
model.add(SimpleRNN(32, return_sequences=True)) # return_sequences Сохраняет размерность для Conv1D
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.build()

print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 176, 128)          3158784   
                                                                 
 masking_4 (Masking)         (None, 176, 128)          0         
                                                                 
 conv1d_5 (Conv1D)           (None, 174, 128)          49280     
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 174, 128)          32896     
                                                                 
 conv1d_6 (Conv1D)           (None, 172, 64)           24640     
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 172, 64)           8256      
                                                                 
 conv1d_7 (Conv1D)           (None, 171, 32)          

In [54]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

Epoch 1/20
30/30 [==============================] - 44s 1s/step - loss: 0.6202 - accuracy: 0.6998 - val_loss: 0.5655 - val_accuracy: 0.7157
Epoch 2/20
130/130 [==============================] - 7s 51ms/step
              precision    recall  f1-score   support

           0       0.74      0.52      0.61      1204
           1       0.82      0.92      0.87      2928

    accuracy                           0.80      4132
   macro avg       0.78      0.72      0.74      4132
weighted avg       0.80      0.80      0.79      4132



### Построение CNN+GRU

In [57]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(32, 4, 8, activation='relu'))
model.add(Dropout(0.5))
model.add(GlobalMaxPool1D(keepdims=True))
model.add(LSTM(32))

model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

preds_proba = model.predict(X_test)
preds = [np.round(l) for l in preds_proba]
print(classification_report([np.round(l) for l in y_test], preds))

Epoch 1/20
30/30 [==============================] - 7s 121ms/step - loss: 0.6539 - accuracy: 0.6904 - val_loss: 0.6025 - val_accuracy: 0.7157
Epoch 2/20
130/130 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

           0       0.68      0.60      0.64      1204
           1       0.84      0.88      0.86      2928

    accuracy                           0.80      4132
   macro avg       0.76      0.74      0.75      4132
weighted avg       0.80      0.80      0.80      4132

